# Ingest Space Station
Demonstrates how to call a REST API to read data and store result in the Databricks Catalog.


In [0]:
import requests

# Get current space station position
position_api_response = requests.get("http://api.open-notify.org/iss-now.json")

print(position_api_response.content)


In [0]:
import json

# Get current number of people in space
api_response = requests.get(
    "http://api.open-notify.org/astros.json" 
)

# Parse the result to JSON.
json_data = json.loads(api_response.content)

print(json_data)


In [0]:
# Convert the people list to a dataframe.
if json_data['message']=="success":
    astro_count = json_data['number']
    astros_df = spark.createDataFrame(json_data['people'])

print("Total astronauts:  " + str(astro_count))
display(astros_df)


In [0]:
# Create the destination schema if needed.  The 'bronze' schema aligns to the raw ingestion tier of the medallion architecture.
spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")

In [0]:
# Store astros_df in a table in the bronze schema
astros_df.write.mode("overwrite").saveAsTable("bronze.astronauts")

In [0]:
# Read back in and confirm storage.  Notice that you only need to specify the folder name.
astros_delta_df = spark.read.table("bronze.astronauts")

display(astros_delta_df)